In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
!pip install arize-phoenix

I0000 00:00:1729545246.983697 6043658 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [19]:
import phoenix as px
px.launch_app()

2024-10-21 17:14:09,804 - 8503332672 - session.py-session:511 - WARNING: Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [20]:
from phoenix.otel import register

tracer_provider = register(
  project_name="my-llm-app", # Default is 'default'
)

2024-10-21 17:14:13,495 - 8503332672 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: my-llm-app
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [21]:
pip install openinference-instrumentation-crewai crewai crewai-tools

I0000 00:00:1729545253.500154 6043658 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
from openinference.instrumentation.crewai import CrewAIInstrumentor

CrewAIInstrumentor().instrument(tracer_provider=tracer_provider)

In [23]:
!pip install openinference-instrumentation-langchain

I0000 00:00:1729545257.552774 6043658 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [24]:
from openinference.instrumentation.langchain import LangChainInstrumentor

LangChainInstrumentor().instrument(tracer_provider=tracer_provider)

2024-10-21 17:14:19,608 - 8503332672 - instrumentor.py-instrumentor:100 - WARNING: Attempting to instrument while already instrumented


In [25]:
from dotenv import load_dotenv
load_dotenv()

import os
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

# You can choose to use a local model through Ollama for example. See https://docs.crewai.com/how-to/LLM-Connections/ for more information.

# os.environ["OPENAI_API_BASE"] = 'http://localhost:11434/v1'
# os.environ["OPENAI_MODEL_NAME"] ='openhermes'  # Adjust based on available model
# os.environ["OPENAI_API_KEY"] ='sk-111111111111111111111111111111111111111111111111'

# You can pass an optional llm attribute specifying what model you wanna use.
# It can be a local model through Ollama / LM Studio or a remote
# model like OpenAI, Mistral, Antrophic or others (https://docs.crewai.com/how-to/LLM-Connections/)
# If you don't specify a model, the default is OpenAI gpt-4o
#
# import os
# os.environ['OPENAI_MODEL_NAME'] = 'gpt-3.5-turbo'
#
# OR
#
# from langchain_openai import ChatOpenAI

search_tool = SerperDevTool()

# Define your agents with roles and goals
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
  verbose=True,
  allow_delegation=False,
  # You can pass an optional llm attribute specifying what model you wanna use.
  # llm=ChatOpenAI(model_name="gpt-3.5", temperature=0.7),
  tools=[search_tool]
)
writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=True
)

# Create tasks for your agents
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
  expected_output="Full blog post of at least 4 paragraphs",
  agent=writer
)

# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=True,
  process = Process.sequential
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)



2024-10-21 17:14:19,698 - 8503332672 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-10-21 17:14:19][DEBUG]: == Working Agent: Senior Research Analyst
 [2024-10-21 17:14:19][INFO]: == Starting Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.


> Entering new CrewAgentExecutor chain...
To conduct a comprehensive analysis of the latest advancements in AI in 2024, I need to gather information on key trends, breakthrough technologies, and potential industry impacts. I will begin by searching for the latest developments in AI for the year 2024.

Action: Search the internet
Action Input: {"search_query": "latest advancements in AI 2024"} 

I encountered an error while trying to use the tool. This was the error: 'latin-1' codec can't encode character '\u201c' in position 0: ordinal not in range(256).
 Tool Search the internet accepts these inputs: Search the internet(search_query: 'string') - A tool that can be used to search the internet with a search_quer

2024-10-21 17:14:35,623 - 8503332672 - __init__.py-__init__:100 - WARNING: Invalid type TaskOutput for attribute 'output.value' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types


Thought: I have repeatedly encountered errors while trying to use the search tool. Since I am unable to gather information through the tool, I will provide the best possible analysis based on my existing knowledge and expertise on AI advancements.

Final Answer:

- **Key Trends in AI 2024:**
  - **Explainable AI (XAI):** Continued focus on making AI systems more transparent and interpretable to users, particularly in high-stakes sectors like healthcare and finance.
  - **AI Ethics and Regulation:** Growing emphasis on ethical AI, with more governments and organizations implementing regulations to ensure AI is used responsibly.
  - **AI for Climate Change:** Increased use of AI to tackle climate change, from optimizing energy usage to modeling climate impacts.
  - **Edge AI:** Accelerated development of AI models that function on edge devices, reducing latency and improving privacy by processing data locally.

- **Breakthrough Technologies:**
  - **Generative AI:** Enhanced capabilities

2024-10-21 17:15:37,929 - 8503332672 - __init__.py-__init__:100 - WARNING: Invalid type TaskOutput for attribute 'output.value' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types


Thought: I have gathered all the necessary insights regarding key AI trends, breakthrough technologies, and potential industry impacts. Now, I can craft an engaging and informative blog post that highlights the significant AI advancements for 2024, making it accessible for a tech-savvy audience.

Final Answer: 

**Exploring the Future: Key AI Advancements to Watch in 2024**

As we step into 2024, the world of artificial intelligence (AI) is buzzing with groundbreaking advancements and transformative potential. These innovations are not only enhancing AI capabilities but are also aligning with societal needs for transparency, ethical responsibility, and sustainability. Here’s a look at some of the most significant AI trends and technologies that are set to redefine industries this year.

**Unveiling AI Trends: A New Era of Transparency and Responsibility**

One of the most fascinating trends in AI is Explainable AI (XAI). As AI systems become more integrated into critical decision-makin